<a href="https://colab.research.google.com/github/jameschartouni/arabic_translation/blob/xlm/XLM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installation step
!pip install transformers
!pip install sentencepiece
!pip install bpemb
#creating the folders 
!mkdir data/
!mkdir data/AD_NMT-master
!mkdir data/train/
!mkdir data/test/
!mkdir data/val/
!mkdir data/vocab
!mkdir data/model

#fetching the pkl files
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW' -O data/AD_NMT-master/english-Arabic-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V8_tp8ZlWUYaX7QQL46t0uSRNrVehSf1' -O data/AD_NMT-master/english-Arabic-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V7X0qtuDIyjTHY0wh-ZNoVwsiF4lId2e' -O data/AD_NMT-master/english-Arabic-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UzL4cOWTMCee83KBUh2QO_H62AFVpDQV' -O data/AD_NMT-master/LAV-MSA-2-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UpfCbkxhztof7dvNjeAs1bHjD4SER6h3' -O data/AD_NMT-master/LAV-MSA-2-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UlAZGtYsSfXzK7hrC_PbxQFqTSXD0DMw' -O data/AD_NMT-master/LAV-MSA-2-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UjDX7cCG2S23SPfSHxSPdVayMTxB5Y16' -O data/AD_NMT-master/Magribi_MSA-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UaVWIqRXo0rxuxDF4KArA4bEK1TaLX3l' -O data/AD_NMT-master/Magribi_MSA-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYvlhdYAdfa4riP_4hn3-IEVd1ZUXVTQ' -O data/AD_NMT-master/Magribi_MSA-train.pkl

     |████████████████████████████████| 645kB 2.8MB/s 
     |████████████████████████████████| 3.8MB 13.3MB/s 
     |████████████████████████████████| 890kB 42.6MB/s 
     |████████████████████████████████| 1.1MB 41.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e9caeedeed17b38e947764859a15408958530cdae0947eb9174b62ca2d1e5736
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
--2020-05-21 15:25:49--  https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW
Resolving docs.google.com (docs.google.com)... 64.233.189.100, 64.233.189.139, 64.233.189.138, ...
Connecting to docs.google.com (docs.google.com)|64.233.189.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cb6al397n247jceacjjva68p918qo

In [0]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
from torch.optim import SGD
from torchtext import data
import pandas as pd
import numpy as np
import pickle
import os, io, glob
import functools
from sklearn.model_selection import train_test_split

from bpemb import BPEmb

import transformers
from transformers import XLMTokenizer, XLMModel
import torchtext
from torchtext.data import Field, BucketIterator
from torchtext.datasets import TranslationDataset


import sentencepiece as spm

In [0]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    data_English_MSA_trainval = pickle.load(handle)

with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    data_English_MSA_test = pickle.load(handle)

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA_both = pickle.load(handle) 




with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    data_LAV_MSA_trainval = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    data_LAV_MSA_test = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA_both = pickle.load(handle) 




with open(file_path + "Magribi_MSA-train.pkl", 'rb') as handle:
    data_Magribi_MSA_trainval = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-test.pkl", 'rb') as handle:
    data_Magribi_MSA_test = pickle.load(handle) 

with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA_both = pickle.load(handle) 
    

In [4]:
print(data_English_MSA_both[0:5])
print(data_LAV_MSA_both[0:5])
print(len(data_English_MSA_trainval))
print(len(data_English_MSA_both))

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش', 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة'], ['روح بوشك و فتول عاليسار', 'اذهب تقدم و استدر يسارا'], ['لا لا لازم انه يكون عندك موضوع ما في اشي', ' لا لا يجب ان يكون لديك موضوع هذا ضروري'], ['اوعي تبعدي من هون بلاش تضيعي ', 'لا تبتعد عن هنا حتى لا تفقد الطريق '], ['قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة', 'اقصد صراحة يا امي انا ايضا كرهته من يوم حضرته مثلما تحضره خالتي كرهته و اصبحت لا اطيقه ابدا']]
9000
10001


##Prepare Datasets

SPM Tokenizer

In [5]:
ls data/AD_NMT-master/

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
#splits the train dataset into train and validation sets, define test set as datafile
eng_msa_train, eng_msa_val = train_test_split(data_English_MSA_trainval, test_size=.2, random_state=22)
eng_msa_test = data_English_MSA_test

lav_msa_train, lav_msa_val = train_test_split(data_LAV_MSA_trainval, test_size=.2, random_state=22)
lav_msa_test = data_LAV_MSA_test

mag_msa_train, mag_msa_val = train_test_split(data_Magribi_MSA_trainval, test_size=.2, random_state=22)
mag_msa_test = data_Magribi_MSA_test

In [7]:
print(len(eng_msa_train))
print(len(eng_msa_val))

print(len(lav_msa_train))
print(len(lav_msa_val))

print(len(mag_msa_train))
print(len(mag_msa_val))

7200
1800
11044
2761
14188
3548


In [0]:
file_path = 'data/'

def pytorch_format(ds, src='en', trg='msa', datatype=''):
    src_formatted = datatype + '_' + src + '_' + trg + '.' + src
    trg_formatted = datatype + '_' + src + '_' + trg + '.' + trg
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as srctxt, open(file_path + datatype + "/" + trg_formatted, 'wt') as trgtxt:
        for i, arr in enumerate(ds):
            srctxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[0] + '\n')
            trgtxt.write(datatype.upper() + '_' + str(i).zfill( len(str(len(ds))) - len(str(i))) + '\\01\\' + arr[1] + '\n')

In [0]:
#splits each language pair file into datasets of single language, to be merged again by the pytorch dataset class later

pytorch_format(eng_msa_train, 'eng', 'msa', 'train')
pytorch_format(eng_msa_val, 'eng', 'msa', 'val')
pytorch_format(eng_msa_test, 'eng', 'msa', 'test')

pytorch_format(lav_msa_train, 'lav', 'msa', 'train')
pytorch_format(lav_msa_val, 'lav', 'msa', 'val')
pytorch_format(lav_msa_test, 'lav', 'msa', 'test')

pytorch_format(mag_msa_train, 'mag', 'msa', 'train')
pytorch_format(mag_msa_val, 'mag', 'msa', 'val')
pytorch_format(mag_msa_test, 'mag', 'msa', 'test')

In [0]:
"""
Testing:
Create a text file with all the vocab available from all sources for each language for SentencePiece to create a library 

TODO: implement bpemb model for MSA vocab/train SPM on larger datasets for dialects
"""

en_vocab = open("data/vocab/eng_vocab.txt", "wt")
msa_vocab = open("data/vocab/msa_vocab.txt", "wt")
lav_vocab = open("data/vocab/lav_vocab.txt", "wt")
mag_vocab = open("data/vocab/mag_vocab.txt", "wt")

MSA_text = ""
EN_text = ""

def create_vocab(file='', src='en_vocab', tgt='msa_vocab'):
  for line in file:
        src_sent = line[0]
        src_words = src_sent.split(" ")
        for count, word in enumerate(src_words):
            src.write(word)
        src.write("\n")
        
        tgt_sent = line[1]
        tgt_words = tgt_sent.split(" ")
        for count, word in enumerate(tgt_words):
            tgt.write(word)
        tgt.write("\n")

create_vocab(data_English_MSA_both, en_vocab, msa_vocab)
create_vocab(data_LAV_MSA_both, lav_vocab, msa_vocab)
create_vocab(data_Magribi_MSA_both, mag_vocab, msa_vocab)

en_vocab.close()
msa_vocab.close()
lav_vocab.close()
mag_vocab.close()

In [0]:
VOCAB_SIZE = 32128
spm.SentencePieceTrainer.train('--input=data/vocab/eng_vocab.txt,data/vocab/msa_vocab.txt,data/vocab/lav_vocab.txt,data/vocab/mag_vocab.txt --model_prefix=data/model/spm --vocab_size=' + str(VOCAB_SIZE))

In [12]:
sp = spm.SentencePieceProcessor()
sp.load('data/model/spm.model')

True

In [13]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[552, 3, 100, 3, 76, 3, 52, 3071]


In [14]:
print(sp.encode_as_pieces('هناك شوكة ناقصة'))
print(sp.encode_as_ids('هناك شوكة ناقصة'))

['▁هناك', '▁', 'شوكة', '▁', 'ناقصة']
[396, 3, 10425, 3, 10999]


In [15]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

['▁This', '▁', 'is', '▁', 'a', '▁', 't', 'est']
[552, 3, 100, 3, 76, 3, 52, 3071]


In [16]:
sp.tokenize("This is a test")

[552, 3, 100, 3, 76, 3, 52, 3071]

Pytorch Dataset

In [0]:
#https://pytorch.org/text/_modules/torchtext/data/field.html
import six

class CustomField(Field):
  
  def __init__(self, tokenize=None, init_token=None, eos_token=None, lower=False):
        Field.__init__(self)
        self.tokenize = tokenize

  def preprocess(self, x):
    #print(x)
    x = sp.tokenize((x.rstrip('\n')))
    return x
  


In [0]:
SRC = CustomField(tokenize = sp,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

TRG = CustomField(tokenize = sp,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = False)

SRC.use_vocab = True
TRG.use_vocab = True

In [58]:
print(SRC.tokenize.tokenize("this is a test"))
callable(SRC.tokenize.tokenize("this is a test"))

[3, 245, 3, 100, 3, 76, 3, 52, 3071]


False

In [0]:
eng_msa_train_dataset = TranslationDataset(path='data/train/train_eng_msa.', exts=('eng', 'msa'), fields=(SRC, TRG))
lav_msa_train_dataset = TranslationDataset(path='data/train/train_lav_msa.', exts=('lav', 'msa'), fields=(SRC, TRG))
mag_msa_train_dataset = TranslationDataset(path='data/train/train_mag_msa.', exts=('mag', 'msa'), fields=(SRC, TRG))

In [0]:
SRC.build_vocab(eng_msa_train_dataset, lav_msa_train_dataset, mag_msa_train_dataset)
TRG.build_vocab(eng_msa_train_dataset, lav_msa_train_dataset, mag_msa_train_dataset)

In [0]:
train_data, valid_data, test_data = eng_msa_train_dataset.splits(path= 'data/', train='train/train_eng_msa', validation='val/val_eng_msa', test='test/test_eng_msa', exts=('.eng', '.msa'),
                                                    fields = (SRC, TRG))

In [77]:
print(train_data[10].src)

[2088, 0, 3554, 280, 8012, 0, 723, 0, 1564, 723, 0, 280, 3, 647, 3, 76, 3, 6305, 3, 2198, 17, 32, 3, 11462, 3, 98, 3, 76, 3, 1463, 1360]


In [0]:
BATCH_SIZE =32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device,
    shuffle=True)